In [1]:
from pyspark.sql import SparkSession


spark = SparkSession \
        .builder \
        .appName('aws') \
        .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1646821801438_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read gdelt v2 file from Google BigQuery, filter >= 2013, and save into parquet file

In [2]:
df = spark.read.parquet('s3://bdcc-final-project-bucket/gdelt-v2.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

621401952

In [4]:
df.createOrReplaceTempView('df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_v2 = (spark.sql("""SELECT *
                      FROM df
                      WHERE Year >= 2013
                      ORDER BY Year DESC
                      """))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_v2.write.parquet('s3://bdcc-final-project-bucket/df_v2.parquet',
                    partitionBy=['Year', 'MonthYear'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read gdelt v1 from AWS S3

In [2]:
df_v1 = spark.read.parquet('s3://bdcc-final-project-bucket/df.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_v1.createOrReplaceTempView('df_v1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
type(df_v1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

# Read gdelt v2 from AWS S3 (transferred from GCP)

In [5]:
df_v2 = spark.read.parquet('s3://bdcc-final-project-bucket/df_v2.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_v2.createOrReplaceTempView('df_v2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
type(df_v2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

# Concatenate
## Get 2013-14 from gdelt v1

In [8]:
df_v1 = (spark.sql("""SELECT *
                      FROM df_v1
                      WHERE Year = 2013 OR Year = 2014
                      ORDER BY Year DESC
                      """))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_v1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: string (nullable = true)
 |-- Actor2Type1Code: string (nullable = true)
 |-- Acto

### Count rows and check per year

In [10]:
df_v1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

75789143

In [11]:
from pyspark.sql.functions import desc

(df_v1.groupby('year')
      .count()
      .orderBy(desc('year'))
      .show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|year|   count|
+----+--------+
|2014|48381955|
|2013|27407188|
+----+--------+

## Get 2015-22 from gdelt v2

In [12]:
df_v2 = (spark.sql("""SELECT *
                      FROM df_v2
                      WHERE Year >= 2015
                      ORDER BY Year DESC
                      """))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_v2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GLOBALEVENTID: long (nullable = true)
 |-- SQLDATE: long (nullable = true)
 |-- FractionDate: double (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: string (nullable = true)
 |-- Actor2Type1Code: string (nullable = true)
 |-- Actor2Ty

As seen above, we have 3 new columns for gdelt v2: `Actor1Geo_ADM2Code`, `Actor2Geo_ADM2Code`, `ActionGeo_ADM2Code`. Because they are not needed in our analysis, we can drop these columns.

### Drop excess columns

In [14]:
columns_to_drop = ['Actor1Geo_ADM2Code', 'Actor2Geo_ADM2Code', 'ActionGeo_ADM2Code']
df_v2 = df_v2.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Count rows and check per year

In [15]:
df_v2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

619651383

In [16]:
(df_v2.groupby('year')
      .count()
      .orderBy(desc('year'))
      .show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
|year|    count|
+----+---------+
|2022| 11233162|
|2021| 58835556|
|2020| 65355708|
|2019| 82320409|
|2018| 94274806|
|2017|105043785|
|2016|114490697|
|2015| 88097260|
+----+---------+

# unionByName all dataframes

In [17]:
from functools import reduce
from pyspark.sql import DataFrame

def union_all(*dfs):
    return reduce(DataFrame.unionByName, dfs)


my_dic = {}
my_dic['df_v1'] = df_v1
my_dic['df_v2'] = df_v2

df_combined = union_all(*my_dic.values())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_combined.write.parquet(
    's3://bdcc-final-project-bucket/df_combined.parquet',
    partitionBy=['Year', 'MonthYear'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check file

In [21]:
df_combined_check = spark.read.parquet(
    's3://bdcc-final-project-bucket/df_combined.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_combined_check.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: string (nullable = true)
 |-- Actor2Type1Code: string (nullable = true)
 |-- Acto

In [27]:
(df_combined_check.groupby('year')
                  .count()
                  .orderBy(desc('year'))
                  .show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
|year|    count|
+----+---------+
|2022| 11233162|
|2021| 58835556|
|2020| 65355708|
|2019| 82320409|
|2018| 94274806|
|2017|105043785|
|2016|114490697|
|2015| 88097260|
|2014| 48381955|
|2013| 27407188|
+----+---------+

In [28]:
df_combined_check.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

695440526

The total must be 695,440,526.

# Filter and save as `war_ukr_rus_w_internal.parquet`

In [ ]:
df_filtered = spark.read.parquet('s3://bdcc-final-project-bucket/df_combined.parquet/')

In [ ]:
df_filtered.printSchema()

In [ ]:
df_filtered.createOrReplaceTempView('df_filtered')

In [ ]:
event_codes_path = 's3://bdcc-lab2-bucket/parquet_files/CAMEO.eventcodes.txt'
desc_eventTab = spark.sparkContext.textFile(event_codes_path)
header = desc_eventTab.first()

# Filter out the header, make sure the rest looks correct
log_txt = desc_eventTab.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split("\t"))
event_desc = temp_var.toDF(header.split("\t"))
print('Selected Event Codes from the Dataset'
      ' and its Corresponding Descriptions')
event_desc.show(10, truncate=False)

# For local temporary view of this DataFrame
event_desc.createOrReplaceTempView('event_desc')

In [ ]:
df_filtered_1 = spark.sql(
    """select distinct *
    from (
        select GLOBALEVENTID,
               Actor1Name,
               Actor2Name,
               Actor1CountryCode,
               Actor2CountryCode,
               AvgTone,
               NumMentions,
               GoldSteinScale,
               EventRootCode,
               EventBaseCode,
               EventCode,
               SQLDATE,
               MonthYear,
               Year,
               SOURCEURL
        from df_filtered
        where (
              (Actor1CountryCode in ('CHN', 'USA', 'GBR',
                                    'FRA', 'RUS', 'UKR') and
               Actor2CountryCode in ('UKR', 'RUS')
              )
              or
              (Actor2CountryCode in ('CHN', 'USA', 'GBR',
                                    'FRA', 'RUS', 'UKR') and
               Actor1CountryCode in ('UKR', 'RUS')
              )
              )
              and
               Actor2CountryCode != '' and
               AvgTone != '' and
               NumMentions != '' and
               GoldSteinScale != '' and
               (rlike(df_filtered.SOURCEURL,
"(?i).*?(war|conflict|aggress|crisis|invade|attack|invasion"
"|weapon|nuclear|military|army|politic|alliance|battle|airforce"
"|marine|combat|fight|soldier|discord|martial|power|violence"
"|peace|negotiat|agree|coalition|ally|aid|sanction|refuge|defen|offen).*?"))
        )
        join event_desc on event_desc.CAMEOEVENTCODE == EventCode
        order by SQLDATE
        """
           )
df_filtered_1.show()

In [ ]:
df_filtered_1.write.parquet('s3://bdcc-final-project-bucket/war_ukr_rus_w_internal.parquet')